Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 10: Run MD simulation in with CybertronFF as potential

In [1]:
import time
import numpy as np
from mindspore import load_checkpoint
from mindspore import context

from mindsponge import Molecule
from mindsponge import Sponge
from mindsponge import set_global_units
from mindsponge.callback import RunInfo, WriteH5MD
from mindsponge.control import LeapFrog
from mindsponge.control import Langevin
from mindsponge.optimizer import UpdaterMD

from cybertron.model import MolCT
from cybertron.readout import AtomwiseReadout
from cybertron.cybertron import CybertronFF

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
set_global_units('A', 'kcal/mol')

In [3]:
atom_types = np.array(
    [[6, 1, 1, 1, 6, 8, 6, 8, 6, 8, 6, 8, 6, 8, 25]], np.int32)
coordinate = np.array([
    [0.782936, -0.21384, 1.940403],
    [0.90026, -1.258313, 2.084498],
    [1.793443, 0.267702, 1.791434],
    [0.161631, 0.247471, 2.702921],
    [-1.775807, 0.660242, 0.992526],
    [-2.573144, 0.82639, 1.806692],
    [-0.793238, 0.551875, -1.559148],
    [-0.922246, 0.719072, -2.702972],
    [1.526357, -0.229486, -0.35567],
    [2.624975, -0.473657, -0.641924],
    [-0.786405, -1.533853, -0.007962],
    [-1.266142, -2.537492, 0.254628],
    [0.394547, 1.910025, 0.468161],
    [0.747036, 3.027445, 0.458565],
    [-0.163356, 0.241977, 0.175396],
])
system = Molecule(atomic_number=atom_types, coordinate=coordinate)

In [4]:
mod = MolCT(
    dim_feature=128,
    num_atom_types=100,
    n_interaction=3,
    n_heads=8,
    max_cycles=1,
    cutoff=10,
    fixed_cycles=True,
    length_unit='A',
)

In [5]:
readout = AtomwiseReadout(
    model=mod,
    dim_output=1,
    activation=mod.activation,
    scale=0.5,
)

In [6]:
potential = CybertronFF(
    model=mod,
    readout=readout,
    atom_types=atom_types,
    length_unit='A',
    energy_unit='kcal/mol',
)

In [7]:
param_file = 'checkpoint_c10.ckpt'
load_checkpoint(param_file, net=potential)

{'model.atom_embedding.embedding_table': Parameter (name=model.atom_embedding.embedding_table, shape=(100, 128), dtype=Float32, requires_grad=True),
 'model.dis_filter.linear.weight': Parameter (name=model.dis_filter.linear.weight, shape=(128, 64), dtype=Float32, requires_grad=True),
 'model.dis_filter.linear.bias': Parameter (name=model.dis_filter.linear.bias, shape=(128,), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.0.weight': Parameter (name=model.dis_filter.residual.nonlinear.mlp.0.weight, shape=(128, 128), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.0.bias': Parameter (name=model.dis_filter.residual.nonlinear.mlp.0.bias, shape=(128,), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.1.weight': Parameter (name=model.dis_filter.residual.nonlinear.mlp.1.weight, shape=(128, 128), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.1.bias': Parameter (name=model.di

In [8]:
opt = UpdaterMD(
    system,
    integrator=LeapFrog(system),
    thermostat=Langevin(system, 300),
    time_step=1e-4,
)

In [9]:
md = Sponge(system, potential, opt)
print(md.calc_energy())

[MindSPONGE] Using fully connected neighbour list (not updated).
[[58.57325]]


In [10]:
cb_h5md = WriteH5MD(system, 'Tutorial_C10.h5md', save_freq=10)
cb_sim = RunInfo(10)

In [11]:
beg_time = time.time()
md.run(1000, callbacks=[cb_sim, cb_h5md])
end_time = time.time()
used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print("Run Time: %02d:%02d:%02d" % (h, m, s))

[MindSPONGE] Started simulation at 2023-03-29 11:00:04
[MindSPONGE] Step: 0, E_pot: 58.57325, E_kin: 0.0, E_tot: 58.57325, Temperature: 0.0
[MindSPONGE] Step: 10, E_pot: 58.53354, E_kin: 0.266128, E_tot: 58.799667, Temperature: 6.377182
[MindSPONGE] Step: 20, E_pot: 58.389404, E_kin: 0.654327, E_tot: 59.04373, Temperature: 15.679529
[MindSPONGE] Step: 30, E_pot: 58.18113, E_kin: 1.0526463, E_tot: 59.233776, Temperature: 25.22439
[MindSPONGE] Step: 40, E_pot: 57.88836, E_kin: 1.4170064, E_tot: 59.305367, Temperature: 33.955494
[MindSPONGE] Step: 50, E_pot: 57.479675, E_kin: 2.1047952, E_tot: 59.58447, Temperature: 50.436863
[MindSPONGE] Step: 60, E_pot: 56.903755, E_kin: 3.097414, E_tot: 60.001167, Temperature: 74.222824
[MindSPONGE] Step: 70, E_pot: 56.200455, E_kin: 3.742854, E_tot: 59.94331, Temperature: 89.6894
[MindSPONGE] Step: 80, E_pot: 55.408195, E_kin: 4.875395, E_tot: 60.283592, Temperature: 116.828285
[MindSPONGE] Step: 90, E_pot: 54.68077, E_kin: 5.6277485, E_tot: 60.308517